In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from decimal import Decimal
from re import sub
import re
import sqlite3
import urllib.request
from selenium.webdriver.common.by import By


In [9]:
rm = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&maxPrice=3000000&sortType=6&propertyTypes=&includeSSTC=true&mustHave=garden&dontShow=newHome%2Cretirement%2CsharedOwnership&furnishTypes=&keywords='
response = get(rm)

In [10]:
print(response)

<Response [200]>


In [11]:
print(response.text[:1000])

<!DOCTYPE html>
<html lang="en-GB" class="is-not-modern property-for-sale channel--buy ">
<head>
    <meta charset="utf-8">
    <title>Properties For Sale in London | Rightmove</title>
    <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>
    <meta name="viewport" content="width=device-width, shrink-to-fit=no, initial-scale=1.0, user-scalable=no"/>
    <meta name="format-detection" content="telephone=no"/>
    <meta name="HandheldFriendly" content="True"/>
    <meta name="description" content="Flats &amp; Houses For Sale in London - Find properties with Rightmove - the UK&#39;s largest selection of properties."/>
    <meta name="referrer" content="origin-when-cross-origin"><link rel="preconnect" href="https://media.rightmove.co.uk:443" crossorigin/>
    <link rel="preconnect" href="//product.rightmove.co.uk" crossorigin/><link rel="shortcut icon" href="/pvw/images/favicons/rebranded/favicon.ico"><link rel="apple-touch-icon" sizes="72x72" href="/pvw/images/favicons/rebranded/apple-

In [13]:
# big soup for search results
big_soup = BeautifulSoup(response.text, 'lxml') 

In [15]:
# get individual property pages from the search results
links, ids = get_links(big_soup)

In [63]:
# scrape text data from individual property pages and save in SQL database
data_tuple = get_prop_data(links, ids)

In [219]:
get_img(links, ids)

No floor plan!
No floor plan!


In [152]:
# download images given the url and file path
def link_to_img(link, id, file_path, i=1):
    file_name = 'image-{}-{}.jpg'.format(id, i)
    full_path = '{}{}'.format(file_path, file_name)
    urllib.request.urlretrieve(link, full_path)
    return None

In [218]:
# a function takes in property links and downloads images and floor plans to local folder
def get_img(links,ids):
    file_path_img = 'images/'
    file_path_fp = 'floorplan/'
    for link, _id in zip(links,ids):
        response = get(link)
        soup = BeautifulSoup(response.text, 'lxml')
        try: 
            fp_link = soup.find('img', src=lambda x: x and "FLP" in x)['src']
            link_to_img(fp_link, _id, file_path_fp)
        except: 
            print('No floor plan!')
        img_links = get_img_links(soup)
        for i, url in enumerate(img_links):
            link_to_img(url, _id, file_path_img, i)
    return None


In [14]:
def get_links(soup):  # return a list of property links and ids from search page
   link_container = soup.find_all('a', {'class': 'propertyCard-headerLink'})
   links = []
   ids = []
   rmpage = 'http://www.rightmove.co.uk'
   for link in link_container:
       links.append(rmpage+link['href'])
       ids.append(re.findall(r'\d+', link['href'])[0])
   return links, ids

In [62]:
def get_prop_data(links, ids): # from search page to property pages and scrape texts and store in SQL database
    keys = []
    title = []
    price = []
    description = []

    for link, _id in zip(links, ids):
        response = get(link)
        soup = BeautifulSoup(response.text, 'lxml')
        keys.append(_id)
        title.append(get_title(soup))
        price.append(get_price(soup))
        description.append(get_desc(soup))
    data_tuple = list(zip(keys, title, price, description))
    # print(data_tuple)
    # sql_insert(data_tuple)
    conn = sqlite3.connect('rightmove.db')
    c = conn.cursor()
    c.executemany(""" INSERT INTO rightmove VALUES (?,?,?,?)""", data_tuple)
    conn.commit()
    conn.close()
    return data_tuple 

In [147]:
def get_img_links(soup):  # return property image links on property page
    links = []
    for tag in soup.find_all('meta', {'property': 'og:image'}):
        links.append(tag['content'])
    return links

In [73]:
def get_title(soup): # return property price on property page
    return soup.find('title').text
    

In [72]:
def get_price(soup): # return property price on property page
    text = soup.find(lambda tag: tag.name =='strong' and '£' in tag.text).text
    price = int(Decimal(sub(r'[^\d.]', '', text)))
    return price


In [71]:
def get_desc(soup): # return the full description on property page
    return soup.find('p', {'itemprop': 'description'}).text.strip()

In [198]:
def get_floorplan(soup): # return the floor plan links on property page
    return soup.find('img', scr=lambda x: x and 'FLP' in x)['src']


In [12]:
response1 = get('http://www.rightmove.co.uk/property-for-sale/property-83446267.html')
soup1 = BeautifulSoup(response1.text, 'lxml')

In [20]:
link = soup1.find('img', src=lambda x: x and "FLP" in x)['src']
print(link)
        # tag.find_all('img', {'alt': 'Floorplan'})

https://media.rightmove.co.uk/dir/132k/131029/83446267/131029_17946450_FLP_01_0000_max_600x600.jpg


In [27]:
print(link)
file_name = 'image-{}-{}.jpg'.format(1, 1)
full_path = '../floorplan/' +file_name
urllib.

https://media.rightmove.co.uk/dir/132k/131029/83446267/131029_17946450_FLP_01_0000_max_600x600.jpg


('../floorplan/image-1-1.jpg', <http.client.HTTPMessage at 0x7ff959dd3af0>)

https://media.rightmove.co.uk/dir/132k/131029/83446267/131029_17946450_FLP_01_0000_max_600x600.jpg


Scrape image

In [103]:
id_list = []
for id in id_container:
    id_list.append(id['id'])
id_list

['prop87847934',
 'prop72418641',
 'prop72418614',
 'prop83201740',
 'prop71823405',
 'prop83201713',
 'prop83201689',
 'prop83201695',
 'prop83201680',
 'prop83201686',
 'prop83201656',
 'prop95815424',
 'prop83201629',
 'prop93971246',
 'prop94402898',
 'prop95815343',
 'prop83201476',
 'prop83201473',
 'prop95815328',
 'prop95815319',
 'prop95815325',
 'prop93963950',
 'prop95815295',
 'prop95815277',
 'prop95815238']

Scrape nseted links

In [11]:

api_key = 'cvvxh2z2qyfvwvcwtweu8rks'
r = get('http://api.zoopla.co.uk/api/v1/property_listings.json?area=London&minimum_beds=4&api_key{}'.format(api_key))

In [12]:
r.status_code

403

In [13]:
r.json

<bound method Response.json of <Response [403]>>

In [3]:
from rightmove_webscraper import RightmoveData

In [28]:
url = "https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E87490&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=3&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false"
rm = RightmoveData(url)


In [32]:
property_links = rm.get_results.url
ids = [re.findall(r'\d+', link)[0] for link in property_links]

In [33]:
def link_to_img(link, id, file_path, i=1):
    file_name = 'image-{}-{}.jpg'.format(id, i)
    full_path = file_path+file_name
    urllib.request.urlretrieve(link, full_path)
    return None

In [36]:
for link, _id in zip(property_links, ids):
    response = get(link)
    soup = BeautifulSoup(response.text, 'lxml')
    file_path_img = '../propertyimages/'
    file_path_fp = '../floorplan/'
    try:
        fp_link = soup.find('img', src=lambda x: x and "FLP" in x)['src']
        print(fp_link)
        link_to_img(fp_link, _id, file_path_fp)
    except Exception as e: print(e)

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable


KeyboardInterrupt: 